# Simple chess

Load the python-chess library and stores the board and pieces.

In [ ]:
pip install python-chess

In [ ]:
import chess
import random

## Define function to check whether the game is over, and to return the reason

In [ ]:
def is_game_over(board):
    """
    Check whether game is over. If it is check for the reason.
    Return: game_over (True/False) and reason (text string)
    """
    
    reason = ''
    game_over = board.is_game_over()
        
    if game_over:
        if board.is_checkmate():
            reason  = 'Checkmate'
        elif board.is_stalemate():
            reason = 'Stalemate'
        elif board.is_insufficient_material:
            reason = 'Insufficient material'
        elif board.is_fivefold_repetition():
            reason = 'Five fold repitition'
        else:
            reason = 'Other'
            
    return game_over, reason

## Define method to evaluate board

This is about the simplest possible way to evaluate a board. Each piece is assigned a value, and the the difference between the players' total pieces value is returned.

In [ ]:
def evaluate(board):
    
    # Create dictionary of value of pieces
    piece_value = {
    'king': 0,
    'pawn': 100,
    'knight': 320,
    'bishop': 330,
    'rook': 500,
    'queen': 900}
    
    # Check for checkmate
    if board.is_checkmate():
        # Check if it is whites turn
        if board.turn:
            # Black wins
            return -9999
        else:
            # White wins
            return 9999
    
    # Check for stalemate
    if board.is_stalemate():
        return 0
    
    # Check that they are sufficient pieces for one player to win 
    if board.is_insufficient_material():
        return 0
    
    # Get number of each type of piece
    wp = len(board.pieces(chess.PAWN, chess.WHITE))
    bp = len(board.pieces(chess.PAWN, chess.BLACK))
    wn = len(board.pieces(chess.KNIGHT, chess.WHITE))
    bn = len(board.pieces(chess.KNIGHT, chess.BLACK))
    wb = len(board.pieces(chess.BISHOP, chess.WHITE))
    bb = len(board.pieces(chess.BISHOP, chess.BLACK))
    wr = len(board.pieces(chess.ROOK, chess.WHITE))
    br = len(board.pieces(chess.ROOK, chess.BLACK))
    wq = len(board.pieces(chess.QUEEN, chess.WHITE))
    bq = len(board.pieces(chess.QUEEN, chess.BLACK))
    
    # Score differences in total pieces
    score = (piece_value['pawn'] * (wp-bp) +
             piece_value['knight'] * (wn-bn) +
             piece_value['bishop'] * (wb-bb) +
             piece_value['rook'] * (wr-br) +
             piece_value['queen'] * (wq-bq))
         
    # Add random 0-1 to score (to randomize equal scoring moves)
    score += random.random()
    
    # Return score (return -score for black; a higher positive score is always
    # best for each player)
    
    score_to_return = score if board.turn else -score
    
    return score_to_return

## Define tree search (minimax) functions

These functions serach forward a given number of moves (depth). Each possible move is scored with minimax, and the best score (maximum value) is chosen.
Here we use a simple minimax method (without pruning).

For a description of minimax see this excellent YouTube video:
https://youtu.be/l-hh51ncgDI

In [ ]:
def scoreboard(board, depthleft, evaluate, maximising_player):
    
    """
    Minimax tree search.
    
    If the search starts from the perspective of the maximising player, the 
    maximum score will be returned.
    
    If the search starts from the perspective of the minimising player, the 
    miniumum score will be returned.
    
    https://youtu.be/l-hh51ncgDI

    """
    
    if depthleft == 0:        
            # Return end leaf
            
            score = evaluate(board)
            return_score = score if maximising_player else -score
            
            return return_score


    if maximising_player:
        
        max_eval = -999999    

        for move in board.legal_moves:
            # Get score for each possible move
            board.push(move)   
            # Recurssive depth-first search. This will follow one path down the 
            # search tree until depthleft == 0. 
            score = scoreboard(
                board, depthleft - 1, evaluate, maximising_player=False )
            # Restore the previous position.
            board.pop()
            # Record new best score if discovered
            if score > max_eval:
                max_eval = score
        
        # Return best score 
        return max_eval
    
    else:
        
        min_eval = 999999
        
        for move in board.legal_moves:
            # Get score for each possible move
            board.push(move)   
            # Recurssive depth-first search. This will follow one path down the 
            # search tree until depthleft == 0. 
            score = scoreboard(
                board, depthleft - 1, evaluate, maximising_player=True )
            # Restore the previous position.
            board.pop()
            # Record new best score if discovered
            if score < min_eval:
                min_eval = score
        
        # Return best score from the starting position given to alphabeta search
        return min_eval        
    

def selectmove(board, depth, evaluate):
    """
    This algorithm loops through all legal moves. From each of these starting
    positions the board is evaluated by minimax tree search from the perspective
    of the opposing player. The opposing player is looking to minimise the board
    score, so the maximum value found in the tree search will be the worst 
    posiiton for the opposing player. This is chosen.
    """
    # Return random choice if depth = 0 
    if depth == 0:
        return random.choice(list(board.legal_moves))
    
    # Set up starting variables
    best_move = chess.Move.null()
    best_value = -99999
    
    # Iterate through legal moves
    for move in board.legal_moves:
        # Play move
        board.push(move)
        # Get value of move (reduce depth by 1, as 1 move already made)
        board_value = scoreboard(
            board, depth-1, evaluate, maximising_player=False)
        # Store move and value if best discovered so far
        if board_value > best_value:
            best_value = board_value
            best_move = move
        # Restore the previous position
        board.pop()
        
    return best_move

## Define function to play the next move in a game

Here we will set up the game so that black will serach forward 3 moves, and white 1. Black should win nearly all the time!

`board.turn` returns `True` if White is playing a turn.

In [ ]:
def next_move(board):
    # Get current player
    player = "White" if board.turn else "Black"
    print('Player: ', player)

    # Make black search ahead more
    depth = 1 if board.turn else 3
    
    # Get move
    mov = selectmove(board, depth, evaluate)
    print ('Move: ', mov)
    
    # Make move
    board.push(mov)
    
    # Print board and score
    print ('Score: ', int(evaluate(board)))
    print ()    
    print (board)
    print ()
    
    # Check for end of game
    game_finished, reason = is_game_over(board)    
    if game_finished:
        print ('GAME OVER!!!')
        print (reason)
    
    # Return
    continue_game = not game_finished
    
    return continue_game

## Play game

Create a loop to play the game.

In [ ]:
def play_game():
    """Game loop"""
    CONTINUE = True
    board = chess.Board()
    while CONTINUE:
        CONTINUE = next_move(board)    
    
# Play game
play_game()